In [1]:
import streamlit as st  
import pandas as pd
import numpy as np
import re
import os
import base64
from openai.embeddings_utils import get_embedding,cosine_similarity
import openai
import time 
from ast import literal_eval

In [6]:
sheet_url = "https://docs.google.com/spreadsheets/d/1JvL9JWgxgR7QdWMqGWoadr6jsvwXKFLEvk9Cz-Ytg0M/edit?usp=sharing"
url_1 = sheet_url.replace('/edit?usp=sharing', '/export?format=csv&gid=0')
df = pd.read_csv(url_1, header=[0])

In [21]:
df

,Posted?,Type,Author,Published?,Website,Title,Subtitle,business,society,science,People,Process,Technology,tags,Entities,Priority,iFrame?,Wave,Ivy
0,NaN,NaN,"Cross, Arena",1,https://sloanreview.mit.edu/article/how-to-cat...,How to catalyze innovation in your organization,Enterprise networks are key,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN
1,NaN,NaN,"Cross, Davenport",1,https://www.robcross.org/wp-content/uploads/20...,Collaborate smarter not harder,Some hints at building the right network dynam...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN
2,NaN,NaN,Gianni Giacomelli,1,https://medium.com/@giannigiacomelli69/the-int...,"Why some quit, and some stay: a surprising take",A new view on the #greatresignation in the #fu...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
3,NaN,NaN,Harvard Business Review,1,https://hbr.org/2021/05/research-dispersed-tea...,"Dispersed teams succeed fast, fail slow",NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN
4,NaN,NaN,Harvard Magazine,1,ttps://www.harvardmagazine.com/2021/11/right-n...,Can slime molds think,On the primitives of intelligence (and if this...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319,NaN,NaN,NaN,NaN,https://www.bitsofwonder.co/p/a-revolution-in-...,Multiscale intelligence in biology,Multi-scale intelligence - in biology.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,https://venturebeat.com/ai/can-we-use-generati...,AI to curate ideas and help decision making,"How do you use AI to curate ideas from people,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
321,NaN,NaN,NaN,NaN,https://www.harvardmagazine.com/node/87458,"The collective bowel, and its role","Is this “the collective bowel”, doing its job ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322,NaN,NaN,NaN,NaN,https://time.com/7012847/saffron-huang-divya-s...,Assemblies of humans vs AI,Can AI's behavior be influenced by an assembly...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df['tags'].value_counts()

Series([], Name: count, dtype: int64)

In [8]:
df.columns

Index(['Posted?', 'Type', 'Author', 'Published?', 'Website', 'Title',
       'Subtitle', 'business', 'society', 'science', 'People', 'Process',
       'Technology', 'tags', 'Entities', 'Priority', 'iFrame?', 'Wave', 'Ivy'],
      dtype='object')

In [14]:
import numpy as np
list(set(df['Author'].dropna().to_list()))


['Richard P. Manna and Dirk Helbing',
 'Azeem Azhar',
 'Amy Webb',
 'UCLA',
 'Jan',
 'Geoff Mulgan',
 'Mar',
 'Tal Schuster Adam Fisch Regina Barzilay',
 'Damon Centola',
 'Bernstein, Shore, Lazer',
 'Rafael Kaufmann, Pranav Gupta, Jacob Taylor',
 'Cross, Arena',
 'Brookings',
 'Wooley, Chabris, Pentland, Hashmi, Malone',
 'Aral Sinan',
 'Fry, Cai, Zhang, Wagner',
 'Lewin-Epstein, O., Aharonov, R. & Hadany',
 'Richard McElreath',
 'Vitalik Buterin',
 'Matt Clancy',
 'Nesta',
 'Boston Consulting Group',
 'Lilach Hadany',
 'Rob Cross, Reb Rebele, and Adam Grant',
 'Feb',
 'Various',
 'Dec',
 'Meslec, Aggarwal, Courseu',
 'Gianni Giacomelli',
 'Jemielniak ',
 'Christoph Riedl, Young Ji Kim, Pranav Guptag, Thomas W. Malone, and Anita Williams Woolley',
 'Harvard Business Review',
 'Painter, Daniels, Laubicher, “Innovations are Disproportionately Likely in the Periphery of a Scientific Network”, 2020',
 'The Economist',
 'Harvard Magazine',
 'NEW',
 'Thomas Malone',
 'Cross, Davenport',
 'J

In [15]:
df['business'] = df['business'].apply(lambda x: float(str(x).split('"')[-1]))

In [17]:
df[df['business'].isin([1])]

,Posted?,Type,Author,Published?,Website,Title,Subtitle,business,society,science,People,Process,Technology,tags,Entities,Priority,iFrame?,Wave,Ivy
0,NaN,NaN,"Cross, Arena",1,https://sloanreview.mit.edu/article/how-to-cat...,How to catalyze innovation in your organization,Enterprise networks are key,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN
1,NaN,NaN,"Cross, Davenport",1,https://www.robcross.org/wp-content/uploads/20...,Collaborate smarter not harder,Some hints at building the right network dynam...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN
2,NaN,NaN,Gianni Giacomelli,1,https://medium.com/@giannigiacomelli69/the-int...,"Why some quit, and some stay: a surprising take",A new view on the #greatresignation in the #fu...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
3,NaN,NaN,Harvard Business Review,1,https://hbr.org/2021/05/research-dispersed-tea...,"Dispersed teams succeed fast, fail slow",NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN
6,NaN,NaN,Matt Clancy,1,https://www.newthingsunderthesun.com/pub/5pua5ge3,An example of successful innovation by distrib...,Can idea generation survive WFH? You bet.,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
7,NaN,Book,Steven Johnson,1,https://www.researchgate.net/publication/33281...,Where Good Ideas Come From,"A classic praise for diversity, and more.",1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,NaN
10,NaN,NaN,NaN,August 15 2022,https://www.washingtonpost.com/technology/2021...,MIT expert on work says any boss who thinks em...,The future of work has collectively-intelligen...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN


In [24]:
!pip3 install cohere

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 248 kB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 31.0 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 38.2 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic-core 2.14.6
    Uninstalling pydantic-core-2.14.6:
      Successfully uninstalled pydantic-core-2.14.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
label-studio 1.10.1 requires jsonschema==3.2.0, but you have jsonschema 4.22.0 which is incompatible.
label-studio 1.10.1 requir

In [28]:
response = co.embed(texts=['hello world'], model='embed-multilingual-light-v3.0', input_type="search_document", embedding_types=["float"])

In [32]:
response.embeddings.float_[0]

[0.013916016,
 0.09539795,
 0.02357483,
 -0.057678223,
 0.085876465,
 0.016708374,
 -0.03302002,
 0.05569458,
 0.043121338,
 0.019958496,
 0.054992676,
 -0.0065345764,
 0.07727051,
 -0.025039673,
 -0.08691406,
 -0.002363205,
 0.09716797,
 -0.0062217712,
 0.0006837845,
 -0.016830444,
 0.049926758,
 -0.013908386,
 -0.057250977,
 0.020858765,
 0.0036678314,
 0.041015625,
 -0.047607422,
 0.010749817,
 0.033813477,
 -0.0637207,
 -0.02810669,
 0.055114746,
 0.029785156,
 -0.013969421,
 0.061584473,
 0.14526367,
 -0.015655518,
 -0.064697266,
 0.048553467,
 -0.015914917,
 -0.030654907,
 0.06390381,
 0.021408081,
 0.07562256,
 0.039215088,
 0.07867432,
 -0.030899048,
 0.03579712,
 -0.12536621,
 0.014305115,
 -0.07122803,
 0.009849548,
 0.014823914,
 0.036621094,
 0.012588501,
 -0.10601807,
 0.020339966,
 -0.052856445,
 -0.052124023,
 0.035339355,
 0.045074463,
 0.12780762,
 -0.026428223,
 0.029876709,
 0.05001831,
 0.013648987,
 0.030166626,
 -0.00667572,
 0.0053520203,
 0.018249512,
 -0.020095